In [1]:
cd "C:/DATA/20170405 in vivo ephys/p2_1"

C:\DATA\20170405 in vivo ephys\p2_1


In [2]:
ls

 Volume in drive C has no label.
 Volume Serial Number is A498-0A2C

 Directory of C:\DATA\20170405 in vivo ephys\p2_1

04/06/2017  05:03 PM    <DIR>          .
04/06/2017  05:03 PM    <DIR>          ..
04/06/2017  05:02 PM             1,788 A1x32-Poly2-5mm-50s-177-A32.prb
04/06/2017  05:03 PM        76,815,360 spont_170405_161708.dat
04/05/2017  04:18 PM        85,862,018 spont_170405_161708.rhd
04/06/2017  05:03 PM         7,201,440 spont_170405_161708.stim
04/05/2017  04:19 PM        84,059,378 spont_170405_161808.rhd
               5 File(s)    253,939,984 bytes
               2 Dir(s)  723,481,272,320 bytes free


In [3]:
import sys
sys.path.append('C://Users/Ginty Lab/My Documents/Github/kwik-tools')
import read_rhd as rhd
import numpy as np
import glob, os

In [4]:

basename = 'spont_170405_'

#batch write .rhd to .dat
for file in glob.glob(basename + '*.rhd'):
    d = rhd.read_rhd(file)
    recordings = np.transpose(d['amplifier_data'])
    stimulation = np.transpose(d['aux_input_data'])
    recordings.tofile(os.path.splitext(file)[0] +'.dat')  #strip .rhd file extension and append .dat
    stimulation.tofile(os.path.splitext(file)[0] +'.stim')

#create .prm file    
files = glob.glob(basename + '*.dat')
prb_file = 'A1x32-Poly2-5mm-50s-177-A32.prb'


with open(basename + '.prm', "w") as text_file:
        text_file.write('experiment_name = \'{0}\' \n'.format(basename))
        text_file.write('prb_file = \'{0}\'\n'.format(prb_file)) 
        text_file.write('traces = dict( \n')
        text_file.write('\traw_data_files={0},\n'.format(files))
        text_file.write('\tvoltage_gain={0}.0,\n'.format(192))  #from Intan RHD2000 documentation
        text_file.write('\tsample_rate={0},\n'.format(d['frequency_parameters']['amplifier_sample_rate']))
        text_file.write('\tn_channels={0},\n'.format(len(d['amplifier_channels'])))
        text_file.write('\tdtype=\'uint16\'\n')
        text_file.write('\t)')

    




Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 1 board ADC channel.
Found 0 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 60.012 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Parsing data...
No missing timestamps in data.
Applying notch filter...
10% done...
20% done...
30% done...
40% done...
50% done...
60% done...
70% done...
80% done...
90% done...
Done!  Elapsed time: 4.2 seconds

Reading Intan Technologies RHD2000 Data File, Version 1.5

Found 32 amplifier channels.
Found 3 auxiliary input channels.
Found 1 supply voltage channel.
Found 1 board ADC channel.
Found 0 board digital input channels.
Found 0 board digital outp